In [1]:
from tfvpr_datasets import VisualPlace
import numpy as np
import os
import glob
import pandas as pd

root = "/mnt/d/datasets/kitti360_scenes/2013_05_28_drive_0000_sync"
gt = os.path.join(root, "meta.csv")
paths = glob.glob(os.path.join(root, "*.png"))

ModuleNotFoundError: No module named 'torch'